<a href="https://colab.research.google.com/github/kokami236/kozinkadai/blob/master/11_17%E5%88%86%E5%89%B2%E7%82%B9%E7%BE%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install open3d tqdm

import open3d as o3d
import numpy as np
import os, random, shutil
from collections import defaultdict
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [3]:
# ================================================
#  Google Drive マウント （永続保存のため）
# ================================================


# ================================================
#  必要ならインストール
# ================================================


# ================================================
#  設定（自分のデータに合わせてここだけ変更）
# ================================================
GT_PLY    = "/content/drive/MyDrive/gakusyu/熊本正解データ.ply"   # 完全点群
HOLE_PLY  = "/content/drive/MyDrive/gakusyu/熊本城外きれい1.ply"    # 欠損点群
OUT_ROOT  = "/content/drive/MyDrive/SeedFormer_PCN_Data"  # ← Drive上の保存先！
CATEGORY  = "castle"

VOXEL_M    = 0.25     # グリッドサイズ（25cm）
PATCH_N    = 4096     # 1パッチの点数
TRAIN_SPLIT= 0.9      # 9:1 で学習/テストに分割
RNG_SEED   = 42

random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

# ================================================
#  保存先フォルダ（PCN形式）
# ================================================
PARTIAL_DIR  = f"{OUT_ROOT}/{CATEGORY}/partial"
COMPLETE_DIR = f"{OUT_ROOT}/{CATEGORY}/complete"

os.makedirs(PARTIAL_DIR, exist_ok=True)
os.makedirs(COMPLETE_DIR, exist_ok=True)

# ================================================
# ① 既存パッチの確認（再開機能）
# ================================================
existing_models = []

for sub in ["partial", "complete"]:
    base = f"{OUT_ROOT}/{CATEGORY}/{sub}"
    train_path = f"{base}/train"
    test_path  = f"{base}/test"
    for path in [train_path, test_path]:
        if not os.path.exists(path): continue
        for m in os.listdir(path):
            if m.startswith("patch_"):
                existing_models.append(m)

# 既存パッチIDの取得
existing_models = sorted(set(existing_models))
existing_ids = set(int(m.split("_")[1]) for m in existing_models)

print("🔎 既存パッチ数:", len(existing_models))
print("↪ 既存パッチID:", list(existing_ids)[:10], "...")

# 次の patch_id を決める
start_patch_id = max(existing_ids) + 1 if existing_ids else 0
print("▶ 再開開始ID:", start_patch_id)

# ================================================
# ② 点群読み込み
# ================================================
gt_pcd   = o3d.io.read_point_cloud(GT_PLY)
hole_pcd = o3d.io.read_point_cloud(HOLE_PLY)
gt_pts   = np.asarray(gt_pcd.points)
hole_pts = np.asarray(hole_pcd.points)

# ================================================
# ③ 原点合わせ
# ================================================
mins = np.minimum(gt_pts.min(axis=0), hole_pts.min(axis=0))
gt_norm   = gt_pts   - mins
hole_norm = hole_pts - mins

# ================================================
# ④ グリッド分割
# ================================================
def grid_keys(pts, voxel):
    return np.floor(pts / voxel).astype(int)

gt_idx   = grid_keys(gt_norm, VOXEL_M)
hole_idx = grid_keys(hole_norm, VOXEL_M)

def group_indices(keys):
    table = defaultdict(list)
    for i, k in enumerate(map(tuple, keys)):
        table[k].append(i)
    return table

gt_table   = group_indices(gt_idx)
hole_table = group_indices(hole_idx)

# ================================================
# ⑤ パッチ分割処理
# ================================================
def slice_into_patches(gt_block, hole_block, patch_n):
    patches = []
    max_n = max(len(gt_block), len(hole_block))
    if max_n == 0:
        return patches

    n_slices = (max_n + patch_n - 1) // patch_n

    gt_ord   = np.random.permutation(len(gt_block))   if len(gt_block)   > 0 else np.array([], dtype=int)
    hole_ord = np.random.permutation(len(hole_block)) if len(hole_block) > 0 else np.array([], dtype=int)

    for s in range(n_slices):
        gt_slice   = gt_block[ gt_ord[s*patch_n:(s+1)*patch_n] ]   if len(gt_block)   > 0 else np.empty((0,3))
        hole_slice = hole_block[hole_ord[s*patch_n:(s+1)*patch_n]] if len(hole_block) > 0 else np.empty((0,3))

        # --- 少ない場合はリピートして4096点に揃える ---
        def pad_to_n(arr, n, base):
            if len(arr) == 0 and len(base) > 0:
                take = np.random.choice(len(base), size=n, replace=True)
                return base[take]
            if len(arr) == 0 and len(base) == 0:
                return None
            if len(arr) < n:
                extra = np.random.choice(len(arr), size=n-len(arr), replace=True)
                arr = np.concatenate([arr, arr[extra]], axis=0)
            elif len(arr) > n:
                arr = arr[:n]
            return arr

        gt_patch   = pad_to_n(gt_slice,   patch_n, gt_block)
        hole_patch = pad_to_n(hole_slice, patch_n, hole_block)
        if gt_patch is None or hole_patch is None:
            continue

        patches.append((gt_patch, hole_patch))
    return patches


# ================================================
# ⑥ 切り分け本番（再開対応）
# ================================================
all_models = list(existing_models)
patch_id = start_patch_id

print("\n🚀 パッチ生成開始（再開機能ON）")

keys_list = list(set(list(gt_table.keys()) + list(hole_table.keys())))
total_keys = len(keys_list)

for ki, key in enumerate(tqdm(keys_list)):
    gt_block   = gt_norm[ gt_table.get(key, []) ]
    hole_block = hole_norm[ hole_table.get(key, []) ]

    if len(gt_block) == 0 and len(hole_block) == 0:
        continue

    # グリッドから複数パッチ生成
    ph_pairs = slice_into_patches(gt_block, hole_block, PATCH_N)

    for (gt_patch_norm, hole_patch_norm) in ph_pairs:

        # すでに生成済みならスキップ（再開機能）
        if patch_id in existing_ids:
            patch_id += 1
            continue

        # 元の座標に戻す
        gt_patch   = gt_patch_norm + mins
        hole_patch = hole_patch_norm + mins

        model_name = f"patch_{patch_id:06d}"

        # 保存（PCN形式）
        for base_dir, pts in [(COMPLETE_DIR, gt_patch), (PARTIAL_DIR, hole_patch)]:
            leaf = f"{base_dir}/{model_name}/{model_name}"
            os.makedirs(leaf, exist_ok=True)
            o3d.io.write_point_cloud(
                f"{leaf}/00.pcd",
                o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pts))
            )

        all_models.append(model_name)
        existing_ids.add(patch_id)
        patch_id += 1

print(f"\n🎉 パッチ作成完了（合計: {len(all_models)}）")

# ================================================
# ⑦ train/test 分割（既存分があれば上書きしない）
# ================================================
print("\n📂 train/test 分割中...")

def ensure_subdirs():
    for sub in ["train", "test"]:
        for base in ["partial", "complete"]:
            path = f"{OUT_ROOT}/{CATEGORY}/{base}/{sub}"
            os.makedirs(path, exist_ok=True)

ensure_subdirs()

# すでに振り分け済みならスキップするためチェック
train_test_done = False
if os.listdir(f"{PARTIAL_DIR}/train"):
    print("✔ 既に train/test 分割済み → スキップ")
    train_test_done = True

if not train_test_done:
    num = len(all_models)
    idx = np.arange(num)
    np.random.shuffle(idx)
    cut = int(num * TRAIN_SPLIT)
    train_ids = set(idx[:cut])
    test_ids  = set(idx[cut:])

    for base in ["partial", "complete"]:
        for i, m in enumerate(all_models):
            src = f"{OUT_ROOT}/{CATEGORY}/{base}/{m}"
            dst_root = "train" if i in train_ids else "test"
            dst = f"{OUT_ROOT}/{CATEGORY}/{base}/{dst_root}/{m}"
            if os.path.exists(dst):
                shutil.rmtree(dst)
            shutil.move(src, dst)

print("\n🎯 完了！")
print("保存場所：", OUT_ROOT)


🔎 既存パッチ数: 0
↪ 既存パッチID: [] ...
▶ 再開開始ID: 0

🚀 パッチ生成開始（再開機能ON）


100%|██████████| 1624/1624 [00:11<00:00, 146.67it/s]



🎉 パッチ作成完了（合計: 277）

📂 train/test 分割中...

🎯 完了！
保存場所： /content/drive/MyDrive/SeedFormer_PCN_Data


In [3]:
# ============================
#  Drive マウント
# ============================
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%cd /content/seedformer/codes
!ls -al


/content/seedformer/codes
total 92
drwxr-xr-x 8 root root  4096 Nov 16 15:25 .
drwxr-xr-x 5 root root  4096 Nov 16 15:25 ..
drwxr-xr-x 2 root root  4096 Nov 16 15:25 Chamfer3D
drwxr-xr-x 3 root root  4096 Nov 16 15:25 datasets
-rw-r--r-- 1 root root 19981 Nov 16 15:25 manager.py
-rw-r--r-- 1 root root 13815 Nov 16 15:25 model.py
drwxr-xr-x 2 root root  4096 Nov 16 15:25 models
drwxr-xr-x 3 root root  4096 Nov 16 15:25 pointnet2_ops_lib
drwxr-xr-x 2 root root  4096 Nov 16 15:25 pointnet_utils
-rw-r--r-- 1 root root 11241 Nov 16 15:25 train_pcn.py
-rw-r--r-- 1 root root 10747 Nov 16 15:25 train_shapenet55.py
drwxr-xr-x 2 root root  4096 Nov 16 15:25 utils


In [1]:
import os, sys

print("🔍 現在の作業ディレクトリ:", os.getcwd())
print("\n🔍 sys.path:")
for p in sys.path:
    print("  ", p)

print("\n📁 カレントディレクトリの中身:")
!ls -al


🔍 現在の作業ディレクトリ: /content

🔍 sys.path:
   /content
   /env/python
   /usr/lib/python312.zip
   /usr/lib/python3.12
   /usr/lib/python3.12/lib-dynload
   
   /usr/local/lib/python3.12/dist-packages
   /usr/lib/python3/dist-packages
   /usr/local/lib/python3.12/dist-packages/IPython/extensions
   /root/.ipython

📁 カレントディレクトリの中身:
total 16
drwxr-xr-x 1 root root 4096 Nov 12 14:30 .
drwxr-xr-x 1 root root 4096 Nov 16 15:34 ..
drwxr-xr-x 4 root root 4096 Nov 12 14:30 .config
drwxr-xr-x 1 root root 4096 Nov 12 14:30 sample_data


In [1]:
# Python 3.7 をインストール
!sudo apt-get install python3.7 python3.7-dev python3.7-distutils
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 2

# python3 コマンドを Python 3.7 に切り替え
!sudo update-alternatives --config python3
# このとき、表示されたリストから 'python3.7' の番号を選択してください。
# 通常は「1」か「2」を選択することになります。

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7 libpython3.7-dev libpython3.7-minimal libpython3.7-stdlib
  python3.7-lib2to3 python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7 libpython3.7-dev libpython3.7-minimal libpython3.7-stdlib
  python3.7 python3.7-dev python3.7-distutils python3.7-lib2to3
  python3.7-minimal
0 upgraded, 9 newly installed, 0 to remove and 41 not upgraded.
Need to get 11.1 MB of archives.
After this operation, 41.8 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-minimal amd64 3.7.17-1+jammy1 [608 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-minimal amd64 3.7.17-1+jammy1 [1,837 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubunt

In [2]:
# Python 3.7 の pip をインストール
!sudo apt-get install python3.7-pip

# 補足: Python 3.7 をメインの python3 として設定していることを確認
# 前回実行したコマンドで、リストから「3.7」を選択している必要があります。
!sudo update-alternatives --config python3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python3.7-pip
E: Couldn't find any package by glob 'python3.7-pip'
E: Couldn't find any package by regex 'python3.7-pip'
There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
  0            /usr/bin/python3.12   2         auto mode
  1            /usr/bin/python3.10   1         manual mode
  2            /usr/bin/python3.12   2         manual mode
* 3            /usr/bin/python3.7    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 3


In [3]:
# 1-1. Python 3.7 専用の get-pip.py をダウンロード
!curl https://bootstrap.pypa.io/pip/3.7/get-pip.py -o get-pip-37.py

# 1-2. Python 3.7 を使用して pip をインストール
!python3.7 get-pip-37.py

# 1-3. 不要なファイルを削除
!rm get-pip-37.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2574k  100 2574k    0     0  3499k      0 --:--:-- --:--:-- --:--:-- 3497k
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 7.8 MB/s eta 0:00:00


In [4]:
%cd /content
!git clone https://github.com/hrzhou2/seedformer.git

/content
Cloning into 'seedformer'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 113 (delta 36), reused 104 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (113/113), 2.45 MiB | 8.50 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [5]:
# 必要なライブラリをインストール
# transforms3d は以前のエラーで必要と判明
!python3.7 -m pip install transforms3d
# requirements.txt の依存関係をインストール (open3d==0.9.0.0 など)
!python3.7 -m pip install -r /content/seedformer/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.6 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 3.10.0 Requires-Python >=3.10; 3.10.0 Requires-Python >=3.8; 3.10.0rc1 Requires-Python >=3.10; 3.10.1 Requires-Python >=3.10; 3.10.3 Requires-Python >=3.10; 3.10.5 Requires-Python >=3.10; 3.10.6 Requires-Python >=3.10; 3.10.7 Requires-Python >=3.10; 3.11.0 Requires-Python >=3.8; 3.12.0 Requires-Python >=3.9; 3.12.1 Requires-Python >=3.9; 3.13.0 Requires-Python >=3.9; 3.14.0 Requires-Python >=3.9; 3.15.0 Requires-Python >=3.10; 3.15.1 Requires-Python >=3.10; 3.6.0 Requires-Python >=3.8; 3.6.0rc1 Requires-Python >=3.8; 3.6.0rc2 Requires-Python >=3.8; 3.6.1 Requires-Python >=3.8; 3.6.2 Requires-Python >=3.8; 3.6.3 Requires-Python >=3.8; 3.7.0 Requires-Python >=3.8; 3.7.0rc1 Requir

In [6]:
# pprint を除外し、残りの依存関係をインストール
!python3.7 -m pip install easydict tqdm h5py matplotlib open3d==0.9.0.0 opencv-python scipy torch==1.8.1

  Using cached easydict-1.13-py3-none-any.whl.metadata (4.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached h5py-3.8.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl.metadata (6.7 kB)
  Using cached open3d-0.9.0.0-cp37-cp37m-manylinux1_x86_64.whl.metadata (3.5 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 111.1 MB/s eta 0:00:00
 

In [7]:
# PointNet++ および Chamfer3D のビルドに使用する CUDA アーキテクチャを上書き
# Colab環境のGPU（T4: 7.5, V100: 7.0）に対応させます。
%env TORCH_CUDA_ARCH_LIST=7.0;7.5

# seedformer の codes ディレクトリへ移動
%cd /content/seedformer/codes


env: TORCH_CUDA_ARCH_LIST=7.0;7.5
/content/seedformer/codes


In [8]:
# pointnet2_ops_lib/setup.py内のハードコードされたCUDAアーキテクチャリストを修正
%cd /content/seedformer/codes/pointnet2_ops_lib
!sed -i 's/3.7+PTX;5.0;6.0;6.1;6.2;7.0;7.5/7.0;7.5/g' setup.py
%cd ..

# Chamfer3D/setup.py内のハードコードされたCUDAアーキテクチャリストを修正
%cd Chamfer3D
# 以前追加された可能性のある import os と os.environ の行を削除
!sed -i '/^import os/d' setup.py
!sed -i '/^os.environ\["TORCH_CUDA_ARCH_LIST"\]/d' setup.py
# CUDAアーキテクチャリストを置換
!sed -i 's/3.7+PTX;5.0;6.0;6.1;6.2;7.0;7.5/7.0;7.5/g' setup.py
%cd ..
# codes ディレクトリに戻る

/content/seedformer/codes/pointnet2_ops_lib
/content/seedformer/codes
/content/seedformer/codes/Chamfer3D
/content/seedformer/codes


In [9]:
# Chamfer3D/setup.py の先頭に os インポートと TORCH_CUDA_ARCH_LIST の設定を追加
%cd /content/seedformer/codes/Chamfer3D
!sed -i '1 a import os' setup.py
!sed -i '2 a os.environ["TORCH_CUDA_ARCH_LIST"] = "7.0;7.5"' setup.py
%cd ..
# codes ディレクトリに戻る

/content/seedformer/codes/Chamfer3D
/content/seedformer/codes


In [10]:
# Chamfer Distance オペレーションをビルド
%cd /content/seedformer/codes/Chamfer3D
!python3.7 setup.py install
%cd ..

/content/seedformer/codes/Chamfer3D
running install
/usr/local/lib/python3.7/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.7/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standar

In [11]:
# seedformer の codes ディレクトリへ移動
%cd /content/seedformer/codes

# 2-1. PointNet++ オペレーションをビルド (pointnet2_ops の解決)
# pointnet2_ops_lib/setup.py を実行します
%cd pointnet2_ops_lib
!python3.7 setup.py install
%cd ..

# 2-2. Chamfer Distance オペレーションをビルド (Chamfer3D の解決)
# Chamfer3D/setup.py を実行します
%cd Chamfer3D
!python3.7 setup.py install
%cd ..


/content/seedformer/codes
/content/seedformer/codes/pointnet2_ops_lib
running install
/usr/local/lib/python3.7/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.7/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/

In [17]:
!python3 --version
# Output should be Python 3.7.x

Python 3.7.17


In [18]:
# opensdを含む、全てのPython依存関係を再インストール
!python3.7 -m pip install transforms3d easydict tqdm h5py matplotlib open3d==0.9.0.0 opencv-python scipy torch==1.8.1

In [21]:
# open3d を強制的に再インストールし、パスをリフレッシュします
!python3.7 -m pip install open3d==0.9.0.0 --force-reinstall

  Using cached open3d-0.9.0.0-cp37-cp37m-manylinux1_x86_64.whl.metadata (3.5 kB)
  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.15-py3-none-any.whl.metadata (1.6 kB)
  Using cached notebook-6.5.7-py3-none-any.whl.metadata (2.5 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (2.1 kB)
  Using cached comm-0.1.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached ipython-7.34.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached traitlets-5.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached jupyterlab_widgets-3.0.16-py3-none-any.whl.metadata (20 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached tornado-6.2-cp37-abi3-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached pyzmq-26.2.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (6.2 kB)
  Using cached argon2_cffi-23.1.0

In [12]:
from train_pcn import PCNConfig, Manager, args
print("✅ train_pcn のインポートに成功しました")

ModuleNotFoundError: No module named 'open3d'

In [28]:
import sys, site

print("Current Python executable:", sys.executable)
print("Current Python version:", sys.version)
print("Current sys.path:")
for p in sys.path:
    print(f"  {p}")

# Attempt to get site-packages paths for Python 3.7
try:
    # This attempts to find site-packages for the *current* python interpreter
    # If python3.7 is the current executable, this should be accurate.
    print("Python 3.7 site-packages (via site.getsitepackages()):")
    for p in site.getsitepackages():
        print(f"  {p}")
except Exception as e:
    print(f"Could not retrieve site-packages: {e}")

print("Attempting to import open3d...")
try:
    import open3d as o3d
    print("✅ open3d imported successfully!")
except ImportError as e:
    print(f"❌ Failed to import open3d: {e}")


Current Python executable: /usr/bin/python3
Current Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Current sys.path:
  /usr/lib/python3.7/site-packages
  /content/seedformer/codes
  /usr/lib/python3.7/site-packages
  /content/seedformer/codes
  /content
  /env/python
  /usr/lib/python312.zip
  /usr/lib/python3.12
  /usr/lib/python3.12/lib-dynload
  
  /usr/local/lib/python3.12/dist-packages
  /usr/lib/python3/dist-packages
  /usr/local/lib/python3.12/dist-packages/IPython/extensions
  /root/.ipython
Python 3.7 site-packages (via site.getsitepackages()):
  /usr/local/lib/python3.12/dist-packages
  /usr/lib/python3/dist-packages
  /usr/lib/python3.12/dist-packages
Attempting to import open3d...
❌ Failed to import open3d: No module named 'open3d'


In [25]:
# 1. /content に移動
%cd /content

# 2. 念のため以前の seedformer を消す
!rm -rf seedformer

# 3. 公式リポジトリを clone
!git clone https://github.com/hrzhou2/seedformer.git

# 4. codes フォルダに移動
%cd /content/seedformer/codes

# 5. Python のモジュール検索パスに追加
import sys
sys.path.insert(0, "/content/seedformer/codes")
sys.path.insert(0, "/usr/lib/python3.7/site-packages") # Add site-packages for Python 3.7

# 6. train_pcn をインポート
from train_pcn import PCNConfig, Manager, args

print("✅ train_pcn のインポートに成功しました")

/content
Cloning into 'seedformer'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 113 (delta 36), reused 104 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (113/113), 2.45 MiB | 22.01 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/seedformer/codes


ModuleNotFoundError: No module named 'open3d'

In [26]:
import sys, os, torch
%cd /content/seedformer/codes

# Python のモジュール検索パスに追加
# `train_pcn`自体と、Python3.7のsite-packagesにインストールされたOpen3Dなどの依存関係を見つけられるようにする
if "/content/seedformer/codes" not in sys.path:
    sys.path.insert(0, "/content/seedformer/codes")
if "/usr/lib/python3.7/site-packages" not in sys.path:
    sys.path.insert(0, "/usr/lib/python3.7/site-packages")

# train_pcn をインポート
from train_pcn import PCNConfig, Manager, args

print("✅ train_pcn のインポートに成功しました")

/content/seedformer/codes


ModuleNotFoundError: No module named 'open3d'

In [1]:
# ============================
#  1) 設定を読み込み
# ============================
cfg = PCNConfig()

# パッチの点数（切り分けで4096点にしたなら 4096）
cfg.CONST.N_INPUT_POINTS = 4096

# ============================
#  2) データセットのパスを書き換え
# ============================
DATA_ROOT = "/content/drive/MyDrive/SeedFormer_PCN_Data"

cfg.DATASETS.SHAPENET.PARTIAL_POINTS_PATH  = f"{DATA_ROOT}/%s/partial/%s/%s/%02d.pcd"
cfg.DATASETS.SHAPENET.COMPLETE_POINTS_PATH = f"{DATA_ROOT}/%s/complete/%s/%s.pcd"

cfg.DATASET.TRAIN_DATASET = 'ShapeNet'
cfg.DATASET.TEST_DATASET  = 'ShapeNet'

# ============================
#  3) ハイパーパラメータ（Colab T4向け）
# ============================
cfg.TRAIN.BATCH_SIZE      = 8      # T4なら4〜8
cfg.TRAIN.N_EPOCHS        = 30     # 最初は短めで様子見
cfg.TRAIN.SAVE_FREQ       = 10
cfg.TRAIN.LEARNING_RATE   = 1e-3
cfg.TRAIN.LR_MILESTONES   = [10, 20, 25]

cfg.CONST.DEVICE = '0'

# ============================
#  4) 出力フォルダ（Driveに保存 → ランタイム切れてもOK）
# ============================
cfg.DIR.OUT_PATH = "/content/drive/MyDrive/SeedFormer_castle_runs"
os.makedirs(cfg.DIR.OUT_PATH, exist_ok=True)

# ============================
#  5) 既存PCNの学習済みモデルを使いたい場合
# ============================
PRETRAINED_CKPT = "/content/seedformer/checkpoints/pcn/ckpt-best.pth"

if os.path.exists(PRETRAINED_CKPT):
    cfg.CONST.WEIGHTS = PRETRAINED_CKPT
    print("Using pretrained:", PRETRAINED_CKPT)
else:
    print("No pretrained model found → scratch training")

# ============================
#  6) Managerで学習開始
# ============================
args.test = False
args.inference = False
args.output = 0
args.pretrained = ""

manager = Manager(
    cfg=cfg,
    args=args,
    train_name="train_pcn_castle",
    device_list=[0] if torch.cuda.is_available() else []
)

manager.run()

print("学習ログとモデルが保存されたフォルダ：")
print(cfg.DIR.OUT_PATH)


Mounted at /content/drive
[Errno 2] No such file or directory: '/content/seedformer/codes'
/content


ModuleNotFoundError: No module named 'train_pcn'